In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
browser = webdriver.Chrome(executable_path='./chromedriver.exe')
#browser.maximize_window()

In [3]:
url = browser.get('https://www.iqair.com/world-air-quality-ranking')
html_text = browser.page_source

m = browser.find_elements_by_class_name('over-x-auto')

m2 = browser.find_elements(By.XPATH, '//button')
for i in m2:
    if i.text == 'More':
        i.click()
        break


In [4]:
html_text = browser.page_source
tree = BeautifulSoup(html_text, 'html.parser')

links = tree.find_all('td',{'class' : 'text-left'})

urls = []
for link in links:
    urls.append('https://www.iqair.com'+link.find('a',{'class' : 'link-primary'})['href'])
#urls #list url 96 thành phố

In [5]:
city_name = []
aqi = []
pm25 = []
pm10 = []
o3 = []
no2 = []
so2 = []
co = []

In [6]:
def is_number(text):
    for i in text:
        if i.isalpha():
            return False
    return True

In [7]:
def check_element_exist(class_name):
    try:
        #aqi = driver.find_element_by_class_name('aqi-value__value')
        aqi = browser.find_element_by_class_name(class_name)
    except NoSuchElementException:
        return False
    return True

In [8]:
for url in urls:
    browser.get(url)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    title = tree.find('h1',{'class' : 'pagetitle__title ng-star-inserted'}).text
    city = title[15:len(title)]
    print(city)
    city_name.append(city)
    
    index = tree.find_all('div', {'class': 'pollutant-history-type-buttons ng-star-inserted'})
    
    for j in index:
        
        x = browser.find_elements_by_class_name('mat-button-wrapper')
        for X in x:
            if 'DAILY' in X.text:
                X.click()
                break
    #lay ra chi so AQI
        if not check_element_exist('aqi-value__value'):
            aqi.append('NULL')
            pm25.append('NULL')
            pm10.append('NULL')
            o3.append('NULL')
            no2.append('NULL')
            so2.append('NULL')
            co.append('NULL')
        else:
            aqi_val = browser.find_element_by_class_name('aqi-value__value')
            print(aqi_val.text)
            aqi.append(aqi_val.text)
            #lay ra danh sach cac button chi so
            button_board = browser.find_element_by_class_name('pollutant-history-type-buttons')
            buttons = button_board.find_elements_by_tag_name('button')
            #bo button AQI
            buttons = buttons[1:]
            for button in buttons:
                print('button name: ', button.text)
                #click vao tung button
                button.click()
                #lay chi so cuoi cung cua bieu do trong button
                chart = browser.find_element_by_class_name('highcharts-series-group')
                cols = chart.find_elements_by_tag_name('rect')
                element_to_hover = cols[-1]
                ActionChains(browser).move_to_element(element_to_hover).perform()
                values = browser.find_elements_by_tag_name('tspan')
                if len(values) == 1:
                    index_value = 0
                else:
                    index_value = values[-2].text
                    if not is_number(index_value):
                        index_value = 0
                print(index_value)
                
                if button.text == 'PM2.5':
                    pm25.append(index_value)
                elif button.text =='PM10':
                    pm10.append(index_value)
                elif button.text == 'O3':
                    o3.append(index_value)
                    print('append')
                elif button.text == 'NO2':
                    no2.append(index_value)
                elif button.text == 'SO2':
                    so2.append(index_value)
                elif button.text == 'CO':
                    co.append(index_value)
                
            len_aqi = len(aqi)
            if len(pm25) < len_aqi:
                pm25.append('NULL')
            if len(pm10) < len_aqi:
                pm10.append('NULL')
            if len(o3) < len_aqi:
                o3.append('NULL')
            if len(no2) < len_aqi:
                no2.append('NULL')
            if len(so2) < len_aqi:
                so2.append('NULL')
            if len(co) < len_aqi:
                co.append('NULL')

Bishkek
354
button name:  PM2.5
140.3
button name:  PM10
2.2
Delhi
178
button name:  PM2.5
142.5
button name:  PM10
245.5
button name:  O3
16.5
append
button name:  NO2
43.3
button name:  SO2
22.1
button name:  CO
1212.3
Kabul
246
button name:  PM2.5
44.2
Dhaka
204
button name:  PM2.5
142
Nur-Sultan
197
button name:  PM2.5
174
button name:  PM10
229.1
button name:  NO2
21.9
button name:  SO2
43.9
button name:  CO
1499.5
Lahore
185
button name:  PM2.5
107.9
Kathmandu
190
button name:  PM2.5
77.6
button name:  PM10
177.7
button name:  O3
46.3
append
Mumbai
187
button name:  PM2.5
113.9
button name:  PM10
250
button name:  O3
12.6
append
button name:  NO2
48.3
button name:  SO2
20.7
button name:  CO
1468.3
Sarajevo
175
button name:  PM2.5
67
button name:  PM10
62.5
button name:  O3
11.8
append
button name:  NO2
40
button name:  SO2
35.2
Ulaanbaatar
122
button name:  PM2.5
48.8
button name:  PM10
93.5
button name:  O3
5.1
append
button name:  NO2
20
button name:  SO2
49.1
button name:  CO


KeyboardInterrupt: 

In [9]:
data = {'City': city_name, 'AQI': aqi, 'PM2.5': pm25, 'PM10': pm10, 'O3': o3, 'NO2': no2, 'SO2':so2, 'CO': co}
df = pd.DataFrame(data)


In [10]:
df

,City,AQI,PM2.5,PM10,O3,NO2,SO2,CO


In [11]:
df.to_csv("data_cityranking.csv")

In [12]:
#df.to_excel('data1.xlsx')